# Exploring the Metadata in our Project

### Questions:
- What metadata can we get from the authors?
- What metadata can we get from the Sequence Read Archive (SRA)?
- Are there duplications between these datasets?
- Which metadata do we need for further analysis?

### Objectives:
- Remove null and NA values from the metadata
- Construct a metadata table for further analysis that uses data from the SRA and authors.

### Keypoints:
- Cleaning metadata and ensuring the data are consistent for grouping samples is important.


### Getting Started

Before we get started you will need to set your netid and then go into the directory for this assignment under bh_class.

In [ ]:
# Change "MY_NETID" to your netid below, and run this cell
# Change project_num to your project number
netid = "bhurwitz"
project_num = 3
project = "project" + str(project_num)

In [ ]:
# Set the working directory and change into this directory
work_dir = "/xdisk/bhurwitz/bh_class/" + netid + "/assignments/04_metadata"
%cd $work_dir

## Section 1 Creating metadata files

We are going to use [MicrobiomeAnalyst](https://www.microbiomeanalyst.ca/MicrobiomeAnalyst/docs/DataFormat.xhtml) to analyze the results of our metagenomic analyses. Before we get started on analyzing data, let's take a look at our project metadata and the requirements for formatting it.

MicrobiomeAnalyst requires the following format:

1. Tab delimited (.txt) format is required for metadata files. 

2. For metadata, sample names are present in columns and metadata types (e.g. depth, temperature) in rows. 

3. Data values should be discrete, qualitative labels (e.g. HIGH, MED, LOW). 

4. Data values cannot not contain Empty cells or NA values. 

5. Metadata names or metadata labels cannot include tab, since these are used to delimit/separate items.

6. Group names should not include any punctuation marks such as spaces, dashes or slashes. If you must, use an underscore - i.e. clinical-group should be clinical_group.

File format: Sample names/IDs are in first column beginning with "#NAME" in first row.

Example
#NAME       SampleType
Sample1     skin        
Sample2     gut
Sample3     skin                                                                                               
Sample4     gut                                               
Sample5     gut
Sample6     gut
Sample7     skin
Sample8     skin

## Section 1 Exploring the data from the authors 

First we will start out by exploring the data from the authors (sample_meta.txt). These are the metadata the authors provided about the samples in the supplemental data in the journal article.

In [ ]:
# What are the column names in our sample_meta.txt file?
# Let's create a list of the column names 
# First, we will get the header row with `head -1`
# Second, we will use translate "tr" to convert tabs to new lines to make it more readable
# Third, we will use "cat -n" to add column numbers we can refer back to.

!head -1 sample_meta.txt | tr '\t' '\n' | cat -n

### Which columns do we need?

Let's explore these metadata to see what data we need. We will go through the columns bit by bit, to decide what to keep and what to toss.

In [ ]:
# Let's check out our metadata using pandas
# First we need to import a few python modules
import sys
from pathlib import Path
import pandas

In [ ]:
# Next let's set the Metadata path
METADATA_PATH = Path(work_dir)

In [ ]:
# Let's convert our sample metadata file into a dataframe in pandas to explore it.
# We are going to look at the first 9 columns that fit in the frame of the notebook.

sample_meta = pandas.read_csv(METADATA_PATH / "sample_meta.txt", sep='\t')
sample_meta.iloc[:, :9] 

#### What do you see in the first 9 columns in the sample metadata file?

1. Cohort (delete): is repeated and doesn't provide any useful informtion. So, we are going to discard this column.

2. ENA_BioProject (delete): is repeated and doesn't provide any useful informtion. So, we are going to discard this column.

3. ENA_SampleAlias (keep): provides a useful unique identifier that identifies the sample by baby/mom id and timepoint. That one is a keeper.

4. ENA_Accession (delete): gives us a unique accession number for the sample, but doesn't group the data in any useful category. So, we don't need this.

5. age(weeks) (keep): gives us information about the week the baby sample came from, but if you have a mother sample this field contains a "NA". Oh no, we aren't allowed to use that. We need to change this to 'm' for mother, so we have a discrete value. 

6. Individual_type (delete): tells us if the sample is from a Baby, Mother, or Father. You might have all Baby samples, and therefore this column is not useful and can be discarded. 

7. ENA_IndividualID (keep) - This is an identifier for the individual being sampled.

8. Family_nb (keep) - This is the family identifier so we can group all of the baby / mother samples      

9. infant_DeliveryMode (keep) - This is the delivery mode.


In [ ]:
# We are going to look at the next 10-14 columns in the sample metadat file

sample_meta = pandas.read_csv(METADATA_PATH / "sample_meta.txt", sep='\t')
sample_meta.iloc[:, 9:14] 

#### What do you see in columns 10-14 in the sample metadata file?

10. intrapartum_ATB (keep) - This field tells us if intrapartum antibotics were given. Let's keep this field.

11. Sampling_before_birth (delete) - This field tells us if the mother's sample was collected before/after birth. We aren't going to use these data. So, we will delete this.

12. Had_antibiotic (delete) - This field tells us if the mother had antibiotics during pregnancy. We are going to ignore these data given that there is a lot of variability, and many teams only have baby samples.
13. infant_HadProbiotics (delete) - This column notes if the infant was given probiotics. This is outside our scope.
14. infant_HadMedication (delete) - This column notes if the infant was given medication. This is outside our scope.


In [ ]:
# We are going to look at columns 15-20 in the sample metadata file.

sample_meta = pandas.read_csv(METADATA_PATH / "sample_meta.txt", sep='\t')
sample_meta.iloc[:, 14:20] 

#### What about columns 13-16 in the sample metadata file? 
15. infant_TempRegMedication_Regular (delete) - This column notes if the infant was given medication to control temperture. This is also outside out scope.
16. infant_TempRegMedication_Temporary (delete) - This column notes if the infant was given medication to control temperture temporarily. This is also outside out scope.
17. infant_VaginalType (delete) - This tells us if the birth was spontaneous or induced if vaginal. This is interesting, but we don't have enough samples to dig into this.

18. infant_CSectionType (delete) - This tells us if the C section was planned or emergency. This is interesting, but we don't have enough samples to dig into this.

19. infant_Months_Breastfed (delete) - This tells us how many months the babies were breastfed. I selected for samples where the babies we breastfed for >9 months, so we are going to ignore this variable.

20. infant_Months_Exclusive (delete) - This tell us how many months the babies were exclusively breastfed. Again, this is interesting but out of scope.

In [ ]:
# Now that we decided which columns we want, let's go get just those columns and put them in a file.

!cut -f3,5,7,8,9,10 sample_meta.txt > new_sample_meta.txt

In [ ]:
# Let's create a pandas dataframe with just the columns we want.
# Notice that I will also create an index on the data frame, this way we can connect these data to 
# other metadata in the SRA metadata file (coming next!).

new_sample_meta = pandas.read_csv(METADATA_PATH / "new_sample_meta.txt", sep='\t', index_col='ENA_SampleAlias')
new_sample_meta

In [ ]:
# Remember that we can't have null values "NaN". If you have "mother" samples you will need to run the code below
# Replace null values with '-1.0' (this will denote that these are from mothers)
new_sample_meta.fillna('-1.0', inplace=True)


In [ ]:
# let's check to see we don't have null values now
new_sample_meta

#### Getting additional metadata from the Sequence Read Archive

The authors did a nice job of giving us metadata in the supplemental data for the article. But, they also submitted metadata to the Sequence Read Archive. Let's check to see if any of these metadata are important.

In [ ]:
# What are the column names in our accession_meta.txt file?
!head -1 accession_meta.txt | tr '\t' '\n' | cat -n

In [ ]:
# Let's look at the data in the accession metadata from the SRA
# Notice that we are using "transpose" to switch rows and columns to see the complete dataset
accession_meta = pandas.read_csv(METADATA_PATH / "accession_meta.txt", sep='\t', index_col='Sample_Name')
accession_meta.transpose()

#### So what additional data do we need for the SRA metadata?

Let's get a few more columns in the SRA data.

Run (column 1): We need this id because it is the ID for the sequence data we are downloading from the SRA.
Host_age (column 22): This tells us the age of the baby (in months) and the mother (in years).
host_sex (column 24): Host sex
Sample_name (column 41) We need this ID to connect (aka join) to the other metadata in sample_meta.


In [ ]:
!cut -f1,22,24,41 accession_meta.txt > new_accession_meta.txt

In [ ]:
# Let's look at the data in the accession metadata from the SRA
# Notice that we are using "transpose" to switch rows and columns to see the complete dataset
new_accession_meta = pandas.read_csv(METADATA_PATH / "new_accession_meta.txt", sep='\t', index_col='Sample_Name')
new_accession_meta

In [ ]:
metadata_joined = new_accession_meta.join(new_sample_meta)
metadata_joined.set_index('Run', inplace=True)

In [ ]:
metadata_joined

In [ ]:
# Group by the 'Category' column
family_ids = metadata_joined.groupby("Family_nb")

# Iterate over the groups and print them
i=0
for name, group in family_ids:
    print(f"Family ID: {name}")
    i = i +1
print(f"Family Count = {i}")

In [ ]:
# Group by the 'infant_DeliveryMode column
result = metadata_joined.groupby("infant_DeliveryMode").size()
print(result)

In [ ]:
# Group by the 'age(weeks)' column
result = metadata_joined.groupby("age(weeks)").size()
print(result)

In [ ]:
# Group by the 'intrapartum_ATB' column
result = metadata_joined.groupby("intrapartum_ATB").size()
print(result)

In [ ]:
metadata_joined.to_csv('all_metadata.txt', sep='\t', index=True)

In [ ]:
!head all_metadata.txt

In [ ]:
!cp ~/be487-fall-2024/assignments/04_metadata/hw04_metadata.ipynb /xdisk/bhurwitz/bh_class/$netid/assignments/04_metadata/hw04_metedata.ipynb